In [ ]:
import pandas as pd
from google.cloud import storage
import os.path
import os
import gzip
import shutil

NROWS = None

client = storage.Client()
bucket = client.get_bucket('facebook-posts')

for blob in bucket.list_blobs():
    if not blob.name.lower().endswith('.gzip'):
        continue
    path = os.path.join(os.getcwd(), blob.name)
    if not os.path.exists(path):
        print('Downloading {0} to {1} ...'.format(blob.name, path))
        with open(path, 'wb') as file_obj:
            blob.download_to_file(file_obj)
        print('Downloaded {0} to {1}!'.format(blob.name, path))
    
    print('Extracting {0}'.format(path))
    with gzip.open(path, 'rb') as f_in, open(path + ".csv", 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
    print('Extracted {0}'.format(path))

In [ ]:
import pandas as pd
posts = pd.DataFrame()
for i in range(40):
    print('./posts-0000000000{:0>2d}.gzip.csv'.format(i))
    p = pd.read_csv('./posts-{:0>12d}.gzip.csv'.format(i))
    p = p[p['type'] == 'link']
    posts = posts.append(p, ignore_index=True)
    print(posts.shape)

In [ ]:
posts

In [ ]:
import pickle
with open('link_posts.pkl', 'wb') as f:
    pickle.dump(posts, f)

In [ ]:
import pickle
with open('link_posts.pkl', 'rb') as f:
    link_posts = pickle.load(f)

In [ ]:
link_posts

In [ ]:
import TextPreprocessing.Language.detector as ld
link_posts['lang_name'] = link_posts['name'].astype(str).apply(ld.detect)
#posts['lang_description'] = posts['description'].astype(str).apply(ld.detect)
#posts['lang_about'] = posts['about'].astype(str).apply(ld.detect)

#%timeit pages['name'].apply(lambda x: ld.detect(x, 'langdetect'))
#%timeit pages['about'].astype(str).apply(lambda x: ld.detect(x))
#%timeit pages['about'].astype(str).apply(lambda x: ld.detect(x, 'langdetect'))

In [ ]:
en_link_posts = link_posts[link_posts['lang_name'] == 'en']

In [ ]:
import pickle
with open('eng_link_posts.pkl', 'wb') as f:
    pickle.dump(en_link_posts, f)

In [ ]:
import pickle
with open('eng_link_posts.pkl', 'rb') as f:
    en_link_posts = pickle.load(f)

In [ ]:
import Utils.GoogleCloud.storage as gcs
gcs.upload('facebook-posts-processed', 'eng_link_posts.pkl', 'eng_link_posts.pkl')

In [ ]:
bucket = client.get_bucket('facebook-pages')
for blob in bucket.list_blobs():
    print("Downloading " + blob.name + " ...")
    with open(blob.name, 'wb') as file_obj:
        blob.download_to_file(file_obj)


In [ ]:
import pandas as pd

NROWS = None

pages = pd.read_csv('./pages.csv', nrows=NROWS)
pages = pages[['id', 'fan_count']]

In [ ]:
result = pd.merge(en_link_posts, pages, left_on='page_id', right_on='id')

In [ ]:
result['share_ratio'] = result['share'] / result['fan_count']

In [ ]:
result['normalized_share_ratio'] = result['share_ratio'] / result['share_ratio'].max()

In [ ]:
result['normalized_share_ratio'].describe()

In [ ]:
data = result[['name', 'normalized_share_ratio']]

In [ ]:
data['name'] = data['name'].astype(str)
data['name']

In [ ]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopword
    Error loading stopword: Package 'stopword' not found in index

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> stopwords
Command 'stopwords' unrecognized

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopwords
      Unzipping corpora/stopw

In [ ]:
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import re

def string_to_words( string ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(string).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

In [ ]:
# Get the number of reviews based on the dataframe column size
num_name = data["name"].size

# Initialize an empty list to hold the clean reviews
clean_train_name = []

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
for i in range( 0, num_name ):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    if( (i+1)%1000 == 0 ):
        print("Review %d of %d\n" % ( i+1, num_name ))
    clean_train_name.append( string_to_words( data["name"][i] ) )

In [ ]:
import pickle
with open('clean_list_post_name.pkl', 'wb') as f:
    pickle.dump(clean_train_name, f)

In [ ]:
import pickle
with open('clean_list_post_name.pkl', 'rb') as f:
    clean_train_name = pickle.load(f)

In [ ]:
import Utils.GoogleCloud.storage as gcs
gcs.upload('facebook-posts-processed', 'clean_list_post_name.pkl', 'clean_list_post_name.pkl')



In [ ]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_name)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

In [ ]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print(vocab)

In [ ]:
import pickle
with open('clean_list_post_name_features.pkl', 'wb') as f:
    pickle.dump(train_data_features, f)

In [ ]:
print("Training the random forest...")
from sklearn.ensemble import RandomForestRegressor

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestRegressor(n_estimators = 1, n_jobs=-1, verbose = 1, max_depth=3) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, data["normalized_share_ratio"] )

In [ ]:
data["normalized_share_ratio"].describe()

In [ ]:
data["good"] = data["normalized_share_ratio"].apply(lambda x: 1 if x >= 0.000190 else 0)

In [ ]:
data["good"] = data["good"].astype(int)
data.describe()

In [ ]:
print("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 1000, n_jobs=-1, verbose = 1, max_depth=5) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, data["good"] )

In [ ]:
forest.feature_importances_

In [ ]:
importances = list(zip(vocab, forest.feature_importances_))
importances.sort(key = lambda t: t[1], reverse=True)
importances

In [ ]:
train_data_features = train_data_features.astype('uint8')

In [ ]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1, max_iter=10, copy_X =False)
clf.fit(train_data_features, data["normalized_share_ratio"])
print(clf.coef_)

In [ ]:
import xgboost as xgb

gbm = xgb.XGBClassifier(max_depth=3, n_estimators=100).fit(train_data_features, data["normalized_share_ratio"])
gbm